## How To Create Custom Quantum Feature Maps

In machine learning, a feature map represents a transformation of data into a higher-dimensional space. However, this can be an expensive computation. Instead, kernel functions can be used to implicitly encode this transformation through the pairwise inner products of data samples. Kernels are a similarity measure over the dataset and are a key component of many machine learning models, for example, support vector machines. A quantum computer can be used to encode classical data into the quantum state space. We call this a quantum feature map. 

In this guide, we will show how to create a custom quantum feature map with trainable parameters, which may be used as input to Qiskit machine learning algorithms such as `QSVC` and `QuantumKernelTrainer`. We will follow four basic steps:

1. Import required Qiskit packages
2. Design the circuit for the quantum feature map
3. Build the circuit with Qiskit
4. Implement the feature map as a `QuantumCircuit` child class

### Import Required Packages

To create a quantum feature map with trainable parameters in Qiskit, there are two basic guidelines.<br>
The quantum feature map should:
 - Be an extension of Qiskit's `QuantumCircuit` class
 - Contain some number of trainable user parameters, `θ`, in addition to parameters designated to input data, `x`

In [1]:
from typing import List, Callable, Union

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

# To visualize circuit creation process
from qiskit.visualization import circuit_drawer

### Design the Circuit
Similarly to classical feature engineering, creating a quantum feature map is a process that strongly depends on the learning problem at hand. In general, we cannot suggest an optimal feature map with no prior knowledge of the learning problem. Instead, we will focus on the basic steps to create a circuit using the Qiskit API. To illustrate, we will build a version of the [covariant feature map](https://github.com/IBM-Quantum-prototypes/quantum-kernel-training/blob/main/qkt/feature_maps/covariant_feature_map.py), which is tailored to a dataset with a particular structure. Check out [this guide](https://github.com/IBM-Quantum-prototypes/quantum-kernel-training/blob/main/docs/background/qkernels_and_data_w_group_structure.ipynb) for more information on covariant quantum kernels.

For this example, the feature map will be built from a circuit containing trainable parameters `θ` followed by a circuit encoding the input data `x`. The trainable parameter of the $i$th qubit corresponds to a rotation around the $y$-axis by an angle `θ[i]`. We follow this by an entanglement layer of controlled-$z$ gates. Finally, we encode two features `x[i], x[i+1]` per qubit using consecutive rotations around the $x$ and $z$ axes.   

### Build the Circuit with Qiskit

First, we instantiate a `QuantumCircuit`  and create the circuit layer with trainable parameters `θ[i]`. Here, we will assume we are given a dataset with 12 features and we encode two features per qubit.

In [2]:
# For a dataset with 12 features; and 2 features per qubit
FEATURE_DIMENSION = 12
NUM_QUBITS = int(FEATURE_DIMENSION / 2)

# Qiskit feature maps should generally be QuantumCircuits or extensions of QuantumCircuit
feature_map = QuantumCircuit(NUM_QUBITS)
user_params = ParameterVector("θ", NUM_QUBITS)

# Create circuit layer with trainable parameters
for i in range(NUM_QUBITS):
    feature_map.ry(user_params[i], feature_map.qubits[i])

print(circuit_drawer(feature_map))

     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├
     ├──────────┤
q_1: ┤ Ry(θ[1]) ├
     ├──────────┤
q_2: ┤ Ry(θ[2]) ├
     ├──────────┤
q_3: ┤ Ry(θ[3]) ├
     ├──────────┤
q_4: ┤ Ry(θ[4]) ├
     ├──────────┤
q_5: ┤ Ry(θ[5]) ├
     └──────────┘


Next, we will define an entanglement scheme (a linear map of controlled-$z$ gates) and create the entanglement layer.

In [3]:
# Linear entanglement
entanglement = [
        [i, i+1]
        for i in range(NUM_QUBITS - 1)
    ]

for source, target in entanglement:
    feature_map.cz(feature_map.qubits[source], feature_map.qubits[target])

feature_map.barrier()

print(circuit_drawer(feature_map))

     ┌──────────┐                ░ 
q_0: ┤ Ry(θ[0]) ├─■──────────────░─
     ├──────────┤ │              ░ 
q_1: ┤ Ry(θ[1]) ├─■──■───────────░─
     ├──────────┤    │           ░ 
q_2: ┤ Ry(θ[2]) ├────■──■────────░─
     ├──────────┤       │        ░ 
q_3: ┤ Ry(θ[3]) ├───────■──■─────░─
     ├──────────┤          │     ░ 
q_4: ┤ Ry(θ[4]) ├──────────■──■──░─
     ├──────────┤             │  ░ 
q_5: ┤ Ry(θ[5]) ├─────────────■──░─
     └──────────┘                ░ 


Finally, we encode two features `x[i], x[i+1]` per qubit using a layer of single-qubit rotations.

In [4]:
input_params = ParameterVector("x", FEATURE_DIMENSION)
for i in range(NUM_QUBITS):
    feature_map.rz(input_params[2 * i + 1], feature_map.qubits[i])
    feature_map.rx(input_params[2 * i], feature_map.qubits[i])

print(circuit_drawer(feature_map))

     ┌──────────┐                ░  ┌──────────┐ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─■──────────────░──┤ Rz(x[1]) ├─┤ Rx(x[0]) ├
     ├──────────┤ │              ░  ├──────────┤ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─■──■───────────░──┤ Rz(x[3]) ├─┤ Rx(x[2]) ├
     ├──────────┤    │           ░  ├──────────┤ ├──────────┤
q_2: ┤ Ry(θ[2]) ├────■──■────────░──┤ Rz(x[5]) ├─┤ Rx(x[4]) ├
     ├──────────┤       │        ░  ├──────────┤ ├──────────┤
q_3: ┤ Ry(θ[3]) ├───────■──■─────░──┤ Rz(x[7]) ├─┤ Rx(x[6]) ├
     ├──────────┤          │     ░  ├──────────┤ ├──────────┤
q_4: ┤ Ry(θ[4]) ├──────────■──■──░──┤ Rz(x[9]) ├─┤ Rx(x[8]) ├
     ├──────────┤             │  ░ ┌┴──────────┤┌┴──────────┤
q_5: ┤ Ry(θ[5]) ├─────────────■──░─┤ Rz(x[11]) ├┤ Rx(x[10]) ├
     └──────────┘                ░ └───────────┘└───────────┘


### Implement the Feature Map as a `QuantumCircuit` Child Class

Most Qiskit algorithms that take feature maps as input require the feature map be a class extension of a `QuantumCircuit`. While there are many ways to do this, we suggest the following approach illustrated with `ExampleFeatureMap` that extends `QuantumCircuit`:

The feature map circuit is created upon instantiation such that
 - Parameters such as feature dimension and entanglement scheme should be specified during initialization
 - In the initialization, `QuantumCircuit.__init__()` is called before the feature map circuit is generated, which ensures all `QuantumCircuit` class fields (e.g. `QuantumCircuit.qubits`) are properly initialized
 - After the `QuantumCircuit` constructor has been called, a class method `_generate_feature_map` generates the feature map circuit

In [5]:
class ExampleFeatureMap(QuantumCircuit):
    """The Example Feature Map circuit"""

    def __init__(
        self,
        feature_dimension: int,
        entanglement: Union[str, List[List[int]], Callable[[int], List[int]]] = None,
        name: str = "ExampleFeatureMap",

    ) -> None:
        """Create a new Example Feature Map circuit.
        Args:
            feature_dimension: The number of features
            entanglement: Entanglement scheme to be used in second layer
            name: Name of QuantumCircuit object

        Raises:
            ValueError: ExampleFeatureMap requires an even number of input features
        """
        if (feature_dimension % 2) != 0:
            raise ValueError(
                """
            Example feature map requires an even number of input features.
                """
            )
        self.feature_dimension = feature_dimension
        self.entanglement = entanglement
        self.user_parameters = None

        # Call the QuantumCircuit initialization
        num_qubits = feature_dimension / 2
        super().__init__(
            num_qubits,
            name=name,
        )

        # Build the feature map circuit
        self._generate_feature_map()

    def _generate_feature_map(self):
        # If no entanglement scheme specified, use linear entanglement
        if self.entanglement is None:
            self.entanglement = [
                [i, i+1]
                for i in range(self.num_qubits - 1)
            ]

        # Vector of data parameters
        input_params = ParameterVector("x", self.feature_dimension)

        user_params = ParameterVector("θ", self.num_qubits)
        # Create an initial rotation layer of trainable parameters
        for i in range(self.num_qubits):
            self.ry(user_params[i], self.qubits[i])

        self.user_parameters = user_params

        # Create the entanglement layer
        for source, target in self.entanglement:
            self.cz(self.qubits[source], self.qubits[target])

        self.barrier()

        # Create a circuit representation of the data group
        for i in range(self.num_qubits):
            self.rz(input_params[2 * i + 1], self.qubits[i])
            self.rx(input_params[2 * i], self.qubits[i])

### Instantiate and Inspect the Example Feature Map

Finally, we will instantiate and inspect an `ExampleFeatureMap` object. We will use `feature_dimension=10` and the default linear entanglement, which should produce a 5-qubit feature map circuit.

In [6]:
feature_map = ExampleFeatureMap(feature_dimension=10)
circuit_drawer(feature_map)

┌──────────┐             ░ ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├─■───────────░─┤ Rz(x[1]) ├┤ Rx(x[0]) ├
     ├──────────┤ │           ░ ├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├─■──■────────░─┤ Rz(x[3]) ├┤ Rx(x[2]) ├
     ├──────────┤    │        ░ ├──────────┤├──────────┤
q_2: ┤ Ry(θ[2]) ├────■──■─────░─┤ Rz(x[5]) ├┤ Rx(x[4]) ├
     ├──────────┤       │     ░ ├──────────┤├──────────┤
q_3: ┤ Ry(θ[3]) ├───────■──■──░─┤ Rz(x[7]) ├┤ Rx(x[6]) ├
     ├──────────┤          │  ░ ├──────────┤├──────────┤
q_4: ┤ Ry(θ[4]) ├──────────■──░─┤ Rz(x[9]) ├┤ Rx(x[8]) ├
     └──────────┘             ░ └──────────┘└──────────┘

In [1]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright